# Introduction to Business Analytics
## Bike sharing trips in New York



Participants:

- Elli Georgiou: s223408
- Maria Katarachia: s213633
- Stavroula Douva: s222652
- Michail-Achillefs Katarachias: s222653
- Dimitris Voukatas: s230148



### Table of Content
- Section 1: **Introduction + Data Analysis \& Visualizations**
- Section 2: **Prediction models**
- Section 3: **Exploratory Component**
- Section 4: **Conclusions**

### Section 1 : Introduction and Data Analysis and Visualizations

Firstly, we import all the libraries that will be used in this project.

In [ ]:
%reset -f

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from time import time
import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.graph_objects as go
import math
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso



import warnings
warnings.filterwarnings('ignore')

#### Data importing

In [ ]:
file_path = 'Trips_2018.csv'

if not os.path.exists('plots'):
    os.makedirs('plots')

with open(file_path, 'r') as f:
    df = pd.read_csv(f)

#### Data preprocessing

Let's take a look at how our dataframe looks like.

In [ ]:
df.info()

In [ ]:
# We remove the first column as it is just an intex and does not contribute to our analysis
df = df.drop(df.columns[0], axis=1)


In [ ]:
# Handling missing values
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df = df[pd.to_numeric(df['start_station_id'], errors='coerce').notnull()]
df = df[pd.to_numeric(df['end_station_id'], errors='coerce').notnull()]


In [ ]:
df['starttime'] = pd.to_datetime(df['starttime'])
df['stoptime'] = pd.to_datetime(df['stoptime'])
df['date'] = df['starttime'].dt.date
df['hour'] = df['starttime'].dt.hour
df['day_of_week'] = df['starttime'].dt.dayofweek

In [ ]:
df = df[df['tripduration'] > 0]
df = df[df['tripduration'] <= 86400] #we improved the dataset by removing trips with negative or unusually lengthy durations, especially those lasting more than 86400 seconds (1 day)
df = df[df['tripduration'] <= df['tripduration'].quantile(.99)] #to focus on more usual user behavior, we excluded trips outside the 99th percentile of trip durations.
df['tripduration'] = (df['stoptime'] - df['starttime']).dt.total_seconds()
df = df[df['birth_year'] >= df['birth_year'].quantile(.01)] #We filtered away records with birth years in the lowest 1st percentile to prevent outliers
df = df[df['gender'].isin([0, 1, 2])]



In [ ]:
df.describe() #we use df.describe to investigate more how our data behave

#### Visualizations

In [ ]:
# We visualize our data to gain useful insights.
#  Distribution of trip durations

fig = px.histogram(df, x='tripduration', nbins=100, title='Distribution of Trip Durations')
fig.update_xaxes(title='Trip Duration (seconds)')
fig.update_yaxes(title='Frequency')
fig.show()



The histogram above depicts the distribution of trip durations, in seconds. As the distribution is right skweed, we can tell that shorter trips are more frequent than longer ones. 

In [ ]:
# Age distribution of the users.

fig = px.histogram(df, x=2019 - df['birth_year'], nbins=100, title='Distribution of User Ages')
fig.update_layout(xaxis_title='Age', yaxis_title='Frequency', showlegend=False)
fig.show()

The age distribution histogram clearly indicates the majority of users are between the ages of 25 and 40. Surprisingly, there is an a rise around the age of 50 which could be due to a number of factors, including a default entry value, a data error, or a large number of people in that age range. We notice a decrease of the distribution for the younger (20 years) and older (>60 years) age groups, reflecting that these groups use the service less.

In [ ]:
gender_counts = df['gender'].value_counts()
gender_counts
labels = ['Male', 'Female', 'Unknown']
colors = ['blue', 'pink', 'gray']

fig = plt.figure(facecolor='white')
plt.pie(gender_counts, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.axis('equal')  
plt.title('Gender Distribution')
plt.show()

The pie chart depicts the gender distribution among our users. Men constitute close to 70% of the population, while women represent about 23%. On the other hand, 8% of the user base is labeled 'Unknown.' This could imply that consumers prefer not to identify their gender, or that gender data was not collected.

We finished the first phase of data preparation successfully, leaving us with a cleaned and improved dataset. We're now ready to move on to the next exciting phase: clustering our data by geographic location.

#### Clustering

In [ ]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df[['start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude', 'birth_year', 'tripduration']])
scaled_df = pd.DataFrame(scaled_df, columns=['start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude', 'birth_year', 'tripduration'])

pca = PCA(n_components=6)
pca.fit(scaled_df)
pca_df = pca.transform(scaled_df)
pca_df = pd.DataFrame(pca.components_, columns=['start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude', 'birth_year', 'tripduration'])


plt.figure(figsize=(10, 10))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.savefig(os.path.join('plots', 'pca_cumulative_explained_variance.png'))
plt.show()

corr_matrix = scaled_df.corr()
corr_matrix['tripduration'].sort_values(ascending=False)

for i in range(len(pca_df)):
    print(f"\nPrincipal Component {i + 1} Loadings:")
    print(pca_df.iloc[i].sort_values(ascending=False))


Our Principal Component Analysis results show that geographic coordinates, particularly start and end station latitudes and longitudes, play an important role in the variance of our data.

#### Elbow method

In order to identify the optimal number of clusters for the clustering algorithm we chose the elbow method.

In [ ]:
#####it takes approximately 30 minutes to run so we run it once and then we commented out####

# coordinates_sample = df[['start_station_latitude', 'start_station_longitude']]
# scaler = StandardScaler()
# coordinates_standardized = scaler.fit_transform(coordinates_sample)

# distortions = []
# K = range(1, 40)
# for k in K:
#     kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
#     kmeans.fit(coordinates_standardized)
#     distortions.append(kmeans.inertia_)

# plt.figure(figsize=(12, 6))
# plt.plot(K, distortions, 'bx-')
# plt.xlabel('Number of clusters (k)')
# plt.ylabel('Distortion (Inertia)')
# plt.title('Elbow Method for Optimal k with Sampled Data')
# plt.show()


The optimal number of clusters appears to be around 5 , meaning that adding more clusters beyond this point does not greatly improve the fineness of our grouping. Therefore we chose to continue with the minimum clusters that we are allowed which are 20.

Preprocessed our location data to handle the outliers

In [ ]:
coordinates_start = df[['start_station_latitude', 'start_station_longitude']]
coordinates_end = df[['end_station_latitude', 'end_station_longitude']]

scaler = StandardScaler()
coordinates_standardized = scaler.fit_transform(coordinates_start)

kmeans = KMeans(n_clusters=20, random_state=42)
kmeans.fit(coordinates_standardized)
df['cluster'] = kmeans.predict(coordinates_standardized)

kmeans = KMeans(n_clusters=20, init='k-means++', random_state=42).fit(coordinates_start)
centers = kmeans.cluster_centers_

df['cluster'] = kmeans.predict(coordinates_start)

We effectively categorized our dataset into 20 distinct clusters by applying the K-means algorithm to our standardized start station coordinates. Each data point was assigned to a single cluster using this method. The clustering helps us identify useful information on station popularity and user behavior trends.

In [ ]:
#Plot the clusters on a map.

def plot_stations_map(stations):
    #First before plotting we have to deal with the outliers
    #The latitude of New York City is approximately between 40.4774 and 45.01585, and the longitude is approximately between -79.76259 and -71.18507.

    lon_min = -79.76259
    lat_min = 40.4774
    lon_max = -71.18507
    lat_max = 45.01585

    # Store the stations that are within the boundaries
    stations = stations[
        (stations['start_station_latitude'] > lat_min) &
        (stations['start_station_latitude'] < lat_max) &
        (stations['start_station_longitude'] > lon_min) &
        (stations['start_station_longitude'] < lon_max)
    ]

    title = 'Citi Bike Stations in New York City'
    fig = px.scatter_mapbox(
        stations,
        lat='start_station_latitude',
        lon='start_station_longitude',
        color='cluster',
        mapbox_style='carto-positron',
        zoom=9,
        width=1000,
        height=600
    )
    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            xanchor='center',
            font=dict(size=20)
        )
    )
    fig.show()


In [ ]:
#Remove the locations that are more than 3 standard deviations from the center of the clusters.
# Calculate the distance between each point and its cluster center
distance = kmeans.transform(coordinates_start)
min_distance = np.min(distance, axis=1) #we get the minimum distance for each point and its cluster index
min_distance_cluster = np.argmin(distance, axis=1)
threshold = 2*np.std(distance,axis=1)
within_threshold = np.argwhere(min_distance < threshold).flatten()

df = df.iloc[within_threshold]#we remove the points that are outside the threshold distance of a cluster center

# Plot the stations with an underlying map of New York City.
plot_stations_map(df)




After clustering the stations, we concentrated on making these groups more relevant by removing outliers, especially, stations that were more than two standard deviations out from their cluster centers. The final result is a map, with each station color-coded to its respective cluster. This map not only depicts the distribution of bike stations around the city, but it also highlights locations with high station density.

#### Cluster with the largest demand

After identifying clusters based on departure stations, the next step is to determine which cluster exhibits the largest demand in terms of the number of pickups.

In [ ]:
# We work with the cluster with the largest demand.

larg_cluster = df['cluster'].value_counts().index[0]

df_cluster = df[df['cluster'] == larg_cluster]

# We want to create two timeseries that will describe for hourly intervals the pick up and the dropoffs counts for the one cluster with the largest demand.

df_cluster

### Section 2: Prediction models

In [ ]:
# Data preproccesing we keep only the ccolumns we need
columns_to_remove = ['start_station_latitude', 'start_station_longitude', 'end_station_latitude','end_station_longitude','bikeid','usertype','birth_year','gender']
df_cluster = df_cluster.drop(columns=columns_to_remove, axis=1)


In [ ]:
# We observe how the new dataframe looks like
df_cluster

#### Pick up  & Drop off Dataframe

Initially we create the pick ups dataframe. We have divided our data into hourly intervals in order to prepare it for analysis and make sure it was best organized for our assessment procedure.

We applied time aggregation so we can forecast the demand on an hourly basis. By selecting the relevant columns only we made sure that our models focus on the most impactful features, which are the times of pickups and dropoffs.

In [ ]:
df_cluster['ride_count'] = 1
hourly_pickups = df_cluster.groupby(pd.Grouper(key='starttime', freq='1H')).agg({'ride_count': 'sum',  }).reset_index()
hourly_dropoff = df_cluster.groupby(pd.Grouper(key='stoptime', freq='1H')).agg({'ride_count': 'sum',  }).reset_index()

hourly_dropoff = hourly_dropoff.rename(columns={'ride_count': 'dropoff_counts'})
hourly_pickups = hourly_pickups.rename(columns={'ride_count': 'pickup_counts'})


hourly_pickups.set_index('starttime', inplace=True)
hourly_dropoff.set_index('stoptime', inplace=True)

In [ ]:
# We plot the pick ups on hourly basis 

fig = px.line(hourly_pickups, x=hourly_pickups.index, y='pickup_counts', title='Hourly Pickup Counts Time Series')
fig.update_layout(
    xaxis_title='Datetime',
    yaxis_title='Pickup Counts',
    hovermode='x',  
    template='plotly_dark'  
)

fig.show()


As we expected from spring till late Autumn where the weather conditions are ideally to rent the bike and especially during the summer period the demand of bike sharing is at its peak.

In [ ]:
fig = px.line(hourly_dropoff, x=hourly_dropoff.index, y='dropoff_counts', title='Hourly Dropoff Counts Time Series')

fig.update_layout(
    xaxis_title='Datetime',
    yaxis_title='Dropoff Counts',
    hovermode='x',  
    template='plotly_dark'  
)
fig.show()

#### Linear Regression Model Fitting

To handle the challenge of predicting bike-sharing demand, we used hourly time aggregation, enabling us to accurately forecast the number of pickups and dropoffs for each hour of the next day.

In [ ]:
hourly_pickups['lag_pickups_24hr'] = hourly_pickups['pickup_counts'].shift(24)
hourly_dropoff['lag_dropoffs_24hr'] = hourly_dropoff['dropoff_counts'].shift(24)

lag_hours_day = 48
hourly_pickups['lag_pickups_2day'] = hourly_pickups['pickup_counts'].shift(lag_hours_day)
hourly_dropoff['lag_dropoffs_2day'] = hourly_dropoff['dropoff_counts'].shift(lag_hours_day)

lag_hours_day= 72
hourly_pickups['lag_pickups_3day'] = hourly_pickups['pickup_counts'].shift(lag_hours_day)
hourly_dropoff['lag_dropoffs_3day'] = hourly_dropoff['dropoff_counts'].shift(lag_hours_day)

lag_hours_day= 96
hourly_pickups['lag_pickups_4day'] = hourly_pickups['pickup_counts'].shift(lag_hours_day)
hourly_dropoff['lag_dropoffs_4day'] = hourly_dropoff['dropoff_counts'].shift(lag_hours_day)

lag_hours_day= 120
hourly_pickups['lag_pickups_5day'] = hourly_pickups['pickup_counts'].shift(lag_hours_day)
hourly_dropoff['lag_dropoffs_5day'] = hourly_dropoff['dropoff_counts'].shift(lag_hours_day)

lag_hours_day= 144
hourly_pickups['lag_pickups_6day'] = hourly_pickups['pickup_counts'].shift(lag_hours_day)
hourly_dropoff['lag_dropoffs_6day'] = hourly_dropoff['dropoff_counts'].shift(lag_hours_day)

lag_hours_day= 168
hourly_pickups['lag_pickups_7day'] = hourly_pickups['pickup_counts'].shift(lag_hours_day)
hourly_dropoff['lag_dropoffs_7day'] = hourly_dropoff['dropoff_counts'].shift(lag_hours_day)

hourly_pickups=hourly_pickups.dropna()
hourly_dropoff=hourly_dropoff.dropna()



After hours of trying to  find the optimal lags to use in our model we conclude that makes sense to take as input for our train model the information from previous days. Towards this we compute the following lags: [24,48,72,96,120,144,168] which translates to 1st day, 2nd day , 3rd day... till day 7 which is the end of the week. In our first attempt we included lag 1 which means we take information for 1 hour ago which does not allign with what our task is. Therefore we believe the best lags to chose is the above approach.

In [ ]:
# Extract day of the week, month, and hour
hourly_pickups['day_of_week'] = hourly_pickups.index.dayofweek
hourly_pickups['month'] = hourly_pickups.index.month
hourly_pickups['hour'] = hourly_pickups.index.hour


# Display the DataFrame
print(hourly_pickups)

Again in our first attempt we did not include the date features such as the month, week and day extracted from startime and stoptime accordingly and therefore as expected the Linear Regression model performed better, suggesting the strong linearity of our data.  In order to avoid overfitting and create a more accurate with better results in terms of r^2 we chose to include also the date features.

In [ ]:
# Extract day of the week, month, and hour
hourly_dropoff['day_of_week'] = hourly_dropoff.index.dayofweek
hourly_dropoff['month'] = hourly_dropoff.index.month
hourly_dropoff['hour'] = hourly_dropoff.index.hour


# Display the DataFrame
hourly_dropoff

Importantly, we followed the task's parameters by training our models on data from January to October and testing them on data from November and December

In [ ]:
#Now we are ready to fit our models and make the predictions:
def fit_and_evaluate(df, target_col, split_date):
    split_date = pd.to_datetime(split_date)
    train = df[df.index < split_date]
    test = df[df.index >= split_date]

    X_train = train.drop(target_col, axis=1)
    y_train = train[target_col]
    X_test = test.drop(target_col, axis=1)
    y_test = test[target_col]

    model = LinearRegression(fit_intercept=False)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    r2 = r2_score(y_test, predictions)
    return r2


split_date = '2018-11-01'
r2_pickups = fit_and_evaluate(hourly_pickups, 'pickup_counts', split_date)
r2_dropoff = fit_and_evaluate(hourly_dropoff, 'dropoff_counts', split_date)

print(f"R² Score for Pickups: {r2_pickups}")
print(f"R² Score for Dropoffs:{r2_dropoff}")

- **Linear Regression for Pickups:** For predicting pickups, the Linear Regression model had an R2 score of  0.6628, which indicates a linear relationship between the features and the pickup counts, with the model accounting for around 66% of the variance in the data. The model's performance is decent, showing that linear parameters such as time of day and historical data work as predictors of bike pickup demand.

- **Linear Regression for Dropoffs:** Similar to pickups, the model scored 0.6616 for dropoffs, showing slightly lower performance in forecasting dropoff numbers than pickups. This means the model's linear relationships, which include time-related features and historical demand, accurately represent the dropoff patterns in bike-sharing activity.

#### Random Forest Model Fitting

In [ ]:
def fit_and_evaluate_with_predictions(df, target_col, split_date):
    split_date = pd.to_datetime(split_date)
    train = df[df.index < split_date]
    test = df[df.index >= split_date]

    X_train = train.drop(target_col, axis=1)
    y_train = train[target_col]
    X_test = test.drop(target_col, axis=1)
    y_test = test[target_col]

    model = RandomForestRegressor(n_estimators=256, max_depth=8, random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    r2 = r2_score(y_test, predictions)
    return r2, y_test, predictions

split_date = '2018-11-01'
r2_pickups, y_test_pickups, predictions_pickups = fit_and_evaluate_with_predictions(hourly_pickups, 'pickup_counts', split_date)
r2_dropoff, y_test_dropoff, predictions_dropoff = fit_and_evaluate_with_predictions(hourly_dropoff, 'dropoff_counts',split_date)

print(f"R² Score for Pickups: {r2_pickups}")
print(f"R² Score for Dropoffs: {r2_dropoff}")

- **Random Forest for Pickups:** The Random Forest model gave a bit higher R2 score of 0.6900 for pickups than the Linear Regression model, indicating a more robust prediction power, which makes sense as we decided to include the date features. 

- **Random Forest for Dropoffs:** The Random Forest model beats the Linear Regression model again for dropoffs with an R2 value of 0.6900. This illustrates the complexity of the dropoff demand patterns, which benefit from the Random Forest's deeper, non-linear modeling approach.

#### Predictions

In [ ]:
#For RandomForest as is the model that performs best
def plot_actual_vs_predicted_scatter(y_test, predictions, title):
    fig = px.scatter(x=y_test, y=predictions, labels={'x': 'Actual', 'y': 'Predicted'}, title=title)
    fig.add_shape(type='line', line=dict(color='black', width=2, dash='dash'), x0=y_test.min(), x1=y_test.max(), y0=y_test.min(), y1=y_test.max())
    fig.update_layout(showlegend=False)
    fig.show()

plot_actual_vs_predicted_scatter(y_test_pickups, predictions_pickups, 'Actual vs Predicted - Pickups')
plot_actual_vs_predicted_scatter(y_test_dropoff, predictions_dropoff, 'Actual vs Predicted - Dropoffs')


- **Pickups plot:** From the actual vs predicted plot we can clearly say that there is a significant linear relationship, implying that the pickup model has good predictive accuracy. However, the density of points near the bottom show that the model is more accurate for smaller demand levels. As the actual values increase, there is some difference, indicating that the model may be less accurate at higher demand levels.

- **Dropoffs plot:** Dropoff predictions follow a similar pattern to pickup predictions, with a good relationship between actual and predicted values.

#### Residual analysis

In [ ]:
mae_pickups = mean_absolute_error(y_test_pickups, predictions_pickups)
mse_pickups = mean_squared_error(y_test_pickups, predictions_pickups)
rmse_pickups = np.sqrt(mse_pickups)

mae_dropoff = mean_absolute_error(y_test_dropoff, predictions_dropoff)
mse_dropoff = mean_squared_error(y_test_dropoff, predictions_dropoff)
rmse_dropoff = np.sqrt(mse_dropoff)

print(f"Pickups - MAE: {mae_pickups}, RMSE: {rmse_pickups}")
print(f"Dropoffs - MAE: {mae_dropoff}, RMSE: {rmse_dropoff}")

residuals_pickups = y_test_pickups - predictions_pickups
residuals_dropoff = y_test_dropoff - predictions_dropoff

fig_res_pickups = px.scatter(x=predictions_pickups, y=residuals_pickups, labels={'x': 'Predictions', 'y': 'Residuals'}, title='Residuals vs Predictions for Pickups')
fig_res_pickups.update_layout(shapes=[dict(type='line', y0=0, y1=0, x0=predictions_pickups.min(), x1=predictions_pickups.max(), line=dict(color='red', dash='dash'))])
fig_res_pickups.show()

fig_res_dropoff = px.scatter(x=predictions_dropoff, y=residuals_dropoff, labels={'x': 'Predictions', 'y': 'Residuals'}, title='Residuals vs Predictions for Dropoffs')
fig_res_dropoff.update_layout(shapes=[dict(type='line', y0=0, y1=0, x0=predictions_dropoff.min(), x1=predictions_dropoff.max(), line=dict(color='red', dash='dash'))])
fig_res_dropoff.show()

fig_hist_pickups = px.histogram(x=residuals_pickups, nbins=50, opacity=0.7, labels={'x': 'Residuals'}, title='Histogram of Residuals for Pickups')
fig_hist_pickups.show()

fig_hist_dropoff = px.histogram(x=residuals_dropoff, nbins=50, opacity=0.7, labels={'x': 'Residuals'}, title='Histogram of Residuals for Dropoffs')
fig_hist_dropoff.show()


We see a continuous concentration of residuals around the zero line in the residual plots for our predictive models, which indicates great forecast accuracy for both pickups and dropoffs. Notably, the spread of residuals increases with prediction magnitude, implying that the model may have limits in capturing peak demand behaviors. The histograms show a right-skewed distribution of residuals, meaning that the model tends to under-predict rather than over-predict, particularly during periods of increasing demand. Our error metrics, MAE and RMSE, measure these differences further, with both indicating a reasonable but improveable model performance. The frequency of bigger mistakes, as shown by RMSE values greater than the MAE, indicates that there are larger errors, which are areas of focus for model improvement.

#### Demand Prediction

In [ ]:
pickups = predictions_pickups[:24]
dropoffs = predictions_dropoff[:24]

net_bikes_per_hour = [np.ceil(dropoffs[i] - pickups[i]) for i in range(len(pickups))]

fig_net_bikes = go.Figure()
fig_net_bikes.add_trace(go.Scatter(x=list(range(len(net_bikes_per_hour))), y=net_bikes_per_hour, mode='lines+markers', name='Net Bikes'))
fig_net_bikes.update_layout(
    title='Net Bikes per Hour',
    xaxis_title='Hour of Day',
    yaxis_title='Net Bikes (Dropoffs - Pickups)',
    xaxis=dict(tickmode='array', tickvals=list(range(len(net_bikes_per_hour))), tickangle=90)  # Rotate x-axis labels vertically
)
fig_net_bikes.show()




In [ ]:
#cumulative net bikes
cumulative_net_bikes = [sum(net_bikes_per_hour[:i+1]) for i in range(len(net_bikes_per_hour))]

fig_cumulative_bikes = go.Figure()
fig_cumulative_bikes.add_trace(go.Scatter(x=list(range(len(cumulative_net_bikes))), y=cumulative_net_bikes, mode='lines+markers', name='Cumulative Net Bikes', line=dict(color='red')))
fig_cumulative_bikes.update_layout(title='Cumulative Net Bikes at Station', xaxis_title='Hour of Day', yaxis_title='Cumulative Net Bikes', xaxis=dict(tickmode='array', tickvals=list(range(len(cumulative_net_bikes)))))
fig_cumulative_bikes.show()

#maximum deficit of bikes (largest negative number)
max_deficit = min(cumulative_net_bikes)
bikes_required = math.ceil(abs(max_deficit))

print(f'The number of bikes required at the start of the day to ensure no shortage is: {bikes_required}')


We used a method based on data to compute the optimal number of bikes that should be available at each station cluster in advance of the next day's demand. What we did is determined the net bikes per hour by taking the projected number of pickups and dropoffs for the first 24 hours of the next day and subtracting the difference between arrivals (dropoffs) and departures (pickups). This step gave us an hourly net flow of bikes at the stations.
The net bikes per hour plot depicts the changing dynamics of bike consumption, telling us as when and where bikes may be in surplus or deficit. We averaged these net values to determine the total net bikes at each station to guarantee that there is never a lack of bikes during the day. The cumulative net bike graph indicated the important times in time when the bike deficit was at its greatest.
We calculated the minimum number of bikes that must be moved to stations overnight by finding the biggest negative value in the cumulative net bikes, which shows the largest deficit. The results of this evaluation, which was based on the realistic goal of reducing any potential bike limited availability, led to the need for 154 bikes to be added to the network at the start of the day.

### Section 3 : Exploratory Component

In the 3rd Section of our porject we examined how much the weather influence the demand of the bikes of New York. The data  contain infromation for the weather of the  year 2018 and they were taken from  https://www.visualcrossing.com/. 

In [ ]:
with open('NYC2018.csv', 'r') as f:
    w = pd.read_csv(f)

w.info()

First step as always is explore and preprocesse our data

In [ ]:
# Dropping irrelevant columns
columns_to_remove = ['preciptype', 'windgust', 'severerisk']

w = w.drop(columns=columns_to_remove)
w.dropna(inplace=True)
w.drop_duplicates(inplace=True)

# Encoding date and time
w['datetime'] = pd.to_datetime(w['datetime'])
w['date'] = w['datetime'].dt.date
w['hour'] = w['datetime'].dt.hour
w['day_of_week'] = w['datetime'].dt.dayofweek

In [ ]:
# Merge the weather data with the pickups and dropoffs dataframes.
w_pickups = pd.merge(w, hourly_pickups, left_on='datetime', right_index=True, how='left')
w_dropoffs = pd.merge(w, hourly_dropoff, left_on='datetime', right_index=True, how='left')
print(w_pickups)

The weather data and the corresponding hourly transportation data were merged based on the 'datetime' column in the weather dataframe and the transportation dataframes' index.

In [ ]:
# Fill pickup counts with the mean of the previous and next hour
w_pickups['pickup_counts'] = w_pickups['pickup_counts'].fillna((w_pickups['pickup_counts'].shift() + w_pickups['pickup_counts'].shift(-1)) / 2)

w_pickups['lag_pickups_24hr'] = w_pickups['lag_pickups_24hr'].fillna((w_pickups['lag_pickups_24hr'].shift() + w_pickups['lag_pickups_24hr'].shift(-1)) / 2)
w_pickups['lag_pickups_2day'] = w_pickups['lag_pickups_2day'].fillna((w_pickups['lag_pickups_2day'].shift() + w_pickups['lag_pickups_2day'].shift(-1)) / 2)
w_pickups['lag_pickups_3day'] = w_pickups['lag_pickups_3day'].fillna((w_pickups['lag_pickups_3day'].shift() + w_pickups['lag_pickups_3day'].shift(-1)) / 2)
w_pickups['lag_pickups_4day'] = w_pickups['lag_pickups_4day'].fillna((w_pickups['lag_pickups_4day'].shift() + w_pickups['lag_pickups_4day'].shift(-1)) / 2)
w_pickups['lag_pickups_5day'] = w_pickups['lag_pickups_5day'].fillna((w_pickups['lag_pickups_5day'].shift() + w_pickups['lag_pickups_5day'].shift(-1)) / 2)
w_pickups['lag_pickups_6day'] = w_pickups['lag_pickups_6day'].fillna((w_pickups['lag_pickups_6day'].shift() + w_pickups['lag_pickups_6day'].shift(-1)) / 2)
w_pickups['lag_pickups_7day'] = w_pickups['lag_pickups_7day'].fillna((w_pickups['lag_pickups_7day'].shift() + w_pickups['lag_pickups_7day'].shift(-1)) / 2)


# Fill dropoff counts with the mean of the previous and next hour
w_dropoffs['dropoff_counts'] = w_dropoffs['dropoff_counts'].fillna((w_dropoffs['dropoff_counts'].shift() + w_dropoffs['dropoff_counts'].shift(-1)) / 2)

w_dropoffs['lag_dropoffs_24hr'] = w_dropoffs['lag_dropoffs_24hr'].fillna((w_dropoffs['lag_dropoffs_24hr'].shift() + w_dropoffs['lag_dropoffs_24hr'].shift(-1)) / 2)
w_dropoffs['lag_dropoffs_2day'] = w_dropoffs['lag_dropoffs_2day'].fillna((w_dropoffs['lag_dropoffs_2day'].shift() + w_dropoffs['lag_dropoffs_2day'].shift(-1)) / 2)
w_dropoffs['lag_dropoffs_3day'] = w_dropoffs['lag_dropoffs_3day'].fillna((w_dropoffs['lag_dropoffs_3day'].shift() + w_dropoffs['lag_dropoffs_3day'].shift(-1)) / 2)
w_dropoffs['lag_dropoffs_4day'] = w_dropoffs['lag_dropoffs_4day'].fillna((w_dropoffs['lag_dropoffs_4day'].shift() + w_dropoffs['lag_dropoffs_4day'].shift(-1)) / 2)
w_dropoffs['lag_dropoffs_5day'] = w_dropoffs['lag_dropoffs_5day'].fillna((w_dropoffs['lag_dropoffs_5day'].shift() + w_dropoffs['lag_dropoffs_5day'].shift(-1)) / 2)
w_dropoffs['lag_dropoffs_6day'] = w_dropoffs['lag_dropoffs_6day'].fillna((w_dropoffs['lag_dropoffs_6day'].shift() + w_dropoffs['lag_dropoffs_6day'].shift(-1)) / 2)
w_dropoffs['lag_dropoffs_7day'] = w_dropoffs['lag_dropoffs_7day'].fillna((w_dropoffs['lag_dropoffs_7day'].shift() + w_dropoffs['lag_dropoffs_7day'].shift(-1)) / 2)



In [ ]:
w_pickups

In [ ]:
w_dropoffs

We used the mean of the nearby hours to fill in the gaps in the pickup and dropoff counts, as well as their lag features, in order to handle missing values in our combined datasets. For accurate analysis, this method aids in preserving the continuity and integrity of the data.

In [ ]:
w_dropoffs.isnull().sum()
w_pickups.isnull().sum()

In [ ]:
# Exclude non-numeric columns from mean calculation
numeric_columns = w_dropoffs.select_dtypes(include=['float64', 'int64']).columns
w_dropoffs[numeric_columns] = w_dropoffs[numeric_columns].fillna(w_dropoffs[numeric_columns].mean())

numeric_columns = w_pickups.select_dtypes(include=['float64', 'int64']).columns
w_pickups[numeric_columns] = w_pickups[numeric_columns].fillna(w_pickups[numeric_columns].mean())

print(w_dropoffs.isnull().sum())
print(w_pickups.isnull().sum())

In [ ]:
pickup_stats = w_pickups['pickup_counts'].describe()
weather_stats = w_pickups[['temp', 'feelslike', 'dew', 'humidity']].describe()

correlation_matrix = w_pickups[['pickup_counts', 'temp', 'feelslike', 'dew', 'humidity']].corr()

print("Pickup Counts Statistics:")
print(pickup_stats)
print("\nWeather Statistics:")
print(weather_stats)
print("\nCorrelation Matrix:")
print(correlation_matrix)

For pickup counts and weather conditions, we performed correlation analysis and descriptive statistics. Significant variability was evident in the pickup counts, with an average of 51.88 and a standard deviation of 45.14. The average temperature and humidity, according to weather statistics, were 13.74°C and 67.65%, respectively. The temperature (both actual and feels like) and pickup counts showed a moderately positive correlation in our correlation matrix, indicating that warmer weather may result in more pickups. It's interesting to note that there was a negative correlation found between pickup counts and humidity, suggesting that higher humidity could lower pickup frequency. This kind of analysis offers insightful information about how weather might affect transportation patterns.

In [ ]:
# Weather Condition Analysis
plt.figure(figsize=(12, 6))
sns.boxplot(x='conditions', y='pickup_counts', data=w_pickups)
plt.title('Pickup Counts by Weather Conditions')
plt.xticks(rotation=45)
plt.show()

The boxplot illustrates how different weather conditions impact the frequency of pickups. When comparing to bad weather conditions like snow or rain, clear weather displays a greater range of pickup counts with a higher median. It's interesting to note that there are less pickups when there is rain and overcast conditions, as well as when there is snow and overcast conditions combined. This could mean that people are reluctant to travel in these situations. The number of pickups seems to be largely unaffected by partly cloudy weather, and a comparatively stable median indicates steady travel patterns. The correlation results are corroborated by this visual analysis, which shows that weather plays a major role in transportation patterns, with clear skies being the most conducive to higher pickup counts.


In [ ]:
correlation_data = w_pickups[['pickup_counts', 'temp', 'feelslike', 'dew', 'humidity']]
correlation_matrix = correlation_data.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

Warmer weather may result in more pickups, as shown by the correlation heatmap in our report, which shows that temperature and "feels like" are somewhat positively correlated with pickup counts. On the other hand, humidity has a minor detrimental effect on pickups, which means that people might decide not to use transportation services when it's more humid. 

In [ ]:
correlation_data = w_pickups.drop(['datetime', 'name', 'conditions', 'icon', 'stations', 'date'], axis=1)
correlation_matrix = correlation_data.corr()

plt.figure(figsize=(15, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

#### Linear Regression Model Fitting

In [ ]:
lag_features = ['lag_pickups_24hr', 'lag_pickups_2day', 'lag_pickups_7day']

X = w_pickups[['temp'] + lag_features]
y = w_pickups['pickup_counts']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

r_squared_train = r2_score(y_train, y_train_pred)
r_squared_test = r2_score(y_test, y_test_pred)
print(f'Training R-squared: {r_squared_train}')
print(f'Test R-squared: {r_squared_test}')

plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred, label='Training Set', alpha=0.5)
plt.scatter(y_test, y_test_pred, label='Test Set', alpha=0.5)
plt.xlabel('Actual Pickup Counts')
plt.ylabel('Predicted Pickup Counts')
plt.title('Actual vs. Predicted Pickup Counts')
plt.legend()

y_combined = pd.concat([y_train, y_test])
y_pred_combined = pd.concat([pd.Series(y_train_pred), pd.Series(y_test_pred)])

plt.subplot(1, 2, 2)
plt.scatter(y_combined, y_pred_combined, s=0.75)
plt.ylabel("Predicted Pickup Counts")
plt.xlabel("Actual Pickup Counts")
plt.plot([min(y_combined), max(y_combined)], [min(y_combined), max(y_combined)], color="red", linestyle='--', linewidth=2)
plt.title("Actual vs Predicted Pickup Counts")
plt.tight_layout()
plt.show()

In [ ]:
# Extract and display the coefficients
feature_names = ['temp'] + lag_features
coefficients = model.coef_
for feature, coef in zip(feature_names, coefficients):
    print(f'{feature}: {coef}')

for feature, coef in zip(feature_names, coefficients):
    if coef > 0:
        print(f"Higher {feature} leads to an increase in bike pickups.")
    else:
        print(f"Higher {feature} leads to a decrease in bike pickups.")

#### Random Forest Regression Model Fitting

In [ ]:
lag_features = ['lag_pickups_24hr', 'lag_pickups_2day', 'lag_pickups_7day']

X = w_pickups[['temp'] + lag_features]
y = w_pickups['pickup_counts']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42)
model.fit(X_train_scaled, y_train)

y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

r_squared_train = r2_score(y_train, y_train_pred)
r_squared_test = r2_score(y_test, y_test_pred)

print(f'Training R-squared: {r_squared_train}')
print(f'Test R-squared: {r_squared_test}')


plt.scatter(y_train, y_train_pred, label='Training Set', alpha=0.5)
plt.scatter(y_test, y_test_pred, label='Test Set', alpha=0.5)
plt.xlabel('Actual Pickup Counts')
plt.ylabel('Predicted Pickup Counts')
plt.title('Actual vs. Predicted Pickup Counts')
plt.legend()
plt.show()

y_combined = pd.concat([y_train, y_test])
y_pred_combined = pd.concat([pd.Series(y_train_pred), pd.Series(y_test_pred)])

plt.scatter(y_combined, y_pred_combined, s=0.75)
plt.ylabel("Predicted Pickup Counts")
plt.xlabel("Actual Pickup Counts")
plt.plot([min(y_combined), max(y_combined)], [min(y_combined), max(y_combined)], color="red", linestyle='--', linewidth=2)
plt.title("Actual vs Predicted Pickup Counts")
plt.show()

#### Lasso

In [ ]:
w_pickups.columns

In [ ]:
lag_features = ['lag_pickups_24hr', 'lag_pickups_2day', 'lag_pickups_7day']

w_pickups['temp_lag'] = w_pickups['temp'].shift(1)
X = w_pickups[lag_features]
y = w_pickups['pickup_counts']

X.dropna(inplace=True)
y = y[X.index]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
lasso = Lasso(random_state=42)
lasso_grid = GridSearchCV(lasso, param_grid, cv=5, scoring='r2')
lasso_grid.fit(X_train_scaled, y_train)
best_alpha = lasso_grid.best_params_['alpha']
lasso_model = Lasso(alpha=best_alpha, random_state=42)
lasso_model.fit(X_train_scaled, y_train)

y_pred = lasso_model.predict(X_test_scaled)

r_squared = r2_score(y_test, y_pred)
print(f'R-squared (Lasso): {r_squared}')


plt.scatter(y_train, y_train_pred, label='Training Set', alpha=0.5)
plt.scatter(y_test, y_test_pred, label='Test Set', alpha=0.5)
plt.xlabel('Actual Pickup Counts')
plt.ylabel('Predicted Pickup Counts')
plt.title('Actual vs. Predicted Pickup Counts')
plt.legend()
plt.show()


y_combined = pd.concat([y_train, y_test])
y_pred_combined = pd.concat([pd.Series(y_train_pred), pd.Series(y_test_pred)])

plt.scatter(y_combined, y_pred_combined, s=0.75)
plt.ylabel("Predicted Pickup Counts")
plt.xlabel("Actual Pickup Counts")
plt.plot([min(y_combined), max(y_combined)], [min(y_combined), max(y_combined)], color="red", linestyle='--', linewidth=2)
plt.title("Actual vs Predicted Pickup Counts")
plt.show()

Firstly, after running three different models, we discovered that RandomForestRegression is the best. Through additional model analysis, we were able to determine that there is a positive correlation between temperature and the demand for bike sharing, as indicated by the coefficient of 11.7453 for the 'temp' variable, which means higher temperatures are linked to an increase in bike pickups, probably as a result of better riding conditions.

Furthermore, the 24-hour lag feature has a coefficient of 57.8205, indicating that the demand from the previous day influences the current day's bike-sharing usage a lot. The coefficients for the 2-day, 7-day, and 24-hour lags are all positive, which shows that previous pickup trends are strong predictors of the current demand.  Compared to the 24-hour lag, the 2-day lag feature shows a decreased but still good, influence on current demand (coefficient of 2.6392). The largest coefficient, 110.1331, appears in the 7-day lag feature, which suggests a strong weekly cycle in the demand for bike sharing.

### Section 4: Conclusion

| Section         | Findings                                                                                                          | Further improvements                         |
|-------------------------------|-----------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------|
| *Data Preproccesing*             | We cleaned both datasets NYC2018 and Trips2018 by handling missing data, performing filtering and visualizing the outcomes.                                    | Based on the given dataset, the preprocessing we performed is adequate. Perhaps more visualization could have provided more insights for further filtering.  |
| *Model Selection*           | We performed Linear Regression and RandomForestRegression and the last one identified as slightly better  based on predictive accuracy and suitability for the data characteristics.  | By adding more features such as the season of the year, the morning/evening rush and checking whether the previous day was a weekend or a special event might have improved our model in terms of R2 accuracy.|
| *EDA / Temperature's Impact*      | Positive correlation between temperature (coefficient: 11.7453) and bike-sharing demand suggests higher demand rates in better weather conditions. Thus, the answer to our EDA research question on whether weather affects rental behavior is affirmative, and the outcomes align with our expectations. Suggests the need for adaptive operational strategies during warmer periods to meet increased demand.          |  Further questions could be addressed in this area, such as whether inaccurate weather forecasts affect bike rentals. Additionally, data from nearby modes of transport could be utilized, and their correlation with riding activity can also be examined.
| *Lagged Features Analysis*  | Strong influence of 24-hour, 2-day, and 7-day lag features on current demand, highlighting consistent daily and weekly patterns.  | Emphasizes the importance of considering recent usage history for accurate demand forecasting and planning. Additionally, data from previous years can be utilized, and features such as 'lags_1year' can be imported for this purpose.|

### Contributon table

| Sections          | Team Member                                                                                           |                                            
|----------------------|--------------------------------------------------------------------------------------------------------|
|  Intoduction    | Elli Georgiou, Maria Katarachia,Stavroula Douva, Michail-Achillefs Katarachias, Dimitris Voukatas     |
| PCA, Elbow method & Clustering   | Michail-Achillefs Katarachias  |
| Prediction Model   | Elli Georgiou,  Michail-Achillefs Katarachias, Dimitris Voukatas   |
| EDA    |Elli Georgiou, Maria Katarachia, Stavroula Douva, Michail-Achillefs Katarachias, Dimitris Voukatas  |
| Conclusions    |     Elli Georgiou, Dimitris Voukatas  |
